## 动态任务规划支持复杂推理与执行，实战构建合规审查系统

## 一、痛点：为什么多数 Agent 做不了真正的“复杂任务”？

设想一个真实的合同合规审查需求：

-  检查 20+ 类风险条款（排他性、自动续约、数据归属等）
-  对比行业标准与历史判例
-  多轮交互澄清模糊表述
-  生成结构化报告并标注依据
  
但现实中，大多数基于 LLM 的 Agent 表现如下：

| 问题 | 表现 |
|------|------|
| 固定流程 | 所有合同走同一套检查流程，无法适配 SaaS/采购/雇佣等类型 |
| 黑盒推理 | 输出结果无中间过程，无法审计“为何漏掉自动续约” |
| 缺乏反馈闭环 | 发现高风险后不会自动追加“竞品对比”或“法条溯源” |
| 工具调用失控 | LLM 幻觉调用不存在的 API，导致执行失败 |

根本原因在于：**缺乏结构化的任务规划能力**。  
LLM 擅长“联想”与“生成”，却不擅长“组织”与“控制”。我们需要一种机制，将人类专家的决策逻辑**显式建模为可执行的任务流**。

## 二、突破：HTN —— 让 Agent 像专家一样“分而治之”

什么是 HTN（Hierarchical Task Network）？

HTN 是一种 **基于知识的任务规划范式**，其核心思想是：

将高层目标逐层分解为可执行的原子动作，并通过“方法（method）”定义分解策略。

它不同于传统规划（如 STRIPS）或纯提示工程（如 Chain-of-Thought），具备三大优势：

| 方法 | 可解释性 | 可控性 | 适应性 |
|------|----------|--------|--------|
| Chain-of-Thought | ⚠️ 依赖模型自由发挥 | ❌ | ⚠️ |
| 强化学习规划 | ❌ 黑箱策略 | ❌ | |
| HTN | 显式任务树 | 规则驱动 | 场景自适应 |

HTN 特别适合法律、金融、医疗等需要**编码专家经验**的场景——因为它允许我们将“资深法务如何审合同”这一过程，**形式化为机器可执行的规则**。

## 三、实战：构建基于 HTN 的合规审查系统

我们将实现一个完整的 HTN Agent，支持：

- 分层任务分解
- 动态方法选择
- 人工干预接口
- 执行日志与记忆系统联动
  
### 1. 定义任务层级结构（Task Hierarchy）

我们设计三层结构：
```python
# tasks.py
TASK_ROOT = {
    "name": "完成合同合规审查",
    "type": "compound",
    "methods": ["method_decompose_and_review"]
}

METHOD_DECOMPOSE_AND_REVIEW = {
    "name": "method_decompose_and_review",
    "task": "完成合同合规审查",
    "subtasks": [
        {"name": "分解合同结构", "type": "primitive", "action": "decompose_contract"},
        {"name": "检查风险条款", "type": "compound", "methods": ["method_check_risks"]},
        {"name": "生成审查报告", "type": "primitive", "action": "generate_report"}
    ]
}

METHOD_CHECK_RISKS = {
    "name": "method_check_risks",
    "task": "检查风险条款",
    "subtasks": [
        {"name": "检查排他性", "type": "primitive", "action": "check_exclusivity"},
        {"name": "检查自动续约", "type": "primitive", "action": "check_auto_renewal"},
        {"name": "检查数据归属", "type": "primitive", "action": "check_data_ownership"},
        {"name": "检查终止条件", "type": "primitive", "action": "check_termination"}
    ]
}
```
 **优势**：任务结构清晰，便于法务专家参与定义与维护。


### 2. 动态规划引擎：运行时选择最优方法

规划器根据上下文（如合同类型、已发现风险）动态选择执行路径：

```python
# planner.py
class HTNPlanner:
    def __init__(self):
        self.methods = {
            "check_exclusivity": self._select_exclusivity_method,
            "validate_with_cases": self._select_validation_method
        }

    def select_method(self, task_name, context):
        selector = self.methods.get(task_name)
        if selector:
            return selector(context)
        return "default_method"

    def _select_exclusivity_method(self, context):
        industry = context.get("contract_type", "generic")
        if industry == "SaaS":
            return "exclusivity_saas_rule"
        elif industry == "Procurement":
            return "exclusivity_procurement_rule"
        else:
            return "exclusivity_generic_rule"

    def _select_validation_method(self, context):
        if context.get("high_risk_found"):
            return "deep_case_analysis"
        else:
            return "quick_reference"
```
 **实现自适应**：
- SaaS 合同 → 重点检查客户锁定条款
- 发现高风险 → 自动触发“深度案例比对”

### 3. 可干预的执行框架（Human-in-the-Loop）

每一步都支持人工干预，确保可控性与合规性：
```python
# executor.py
import time

class HTNExecutor:
    def __init__(self, planner, tools, monitor):
        self.planner = planner
        self.tools = tools
        self.monitor = monitor

    def execute_task(self, task, context):
        print(f"\n[PLANNING] 即将执行: {task['name']}")

        # 选择方法
        method_name = self.planner.select_method(task['name'], context)
        print(f"[SELECT] 使用方法: {method_name}")

        # 人工干预点
        override = self.ask_for_override(task, context)
        if override:
            method_name = override
            print(f"[OVERRIDE] 已由用户替换为: {method_name}")

        # 执行
        result = self.run_method(method_name, context)
        self.monitor.log_step(task['name'], "success", result.get("confidence", 0.8))

        return result

    def ask_for_override(self, task, context):
        # 模拟人工干预（生产环境可接入审批流）
        if task['name'] == "生成审查报告" and context.get("high_risk_found"):
            user_input = input(f"  高风险合同！是否跳过报告生成？(y/n): ")
            if user_input.lower() == 'y':
                return "escalate_to_human"
        return None

    def run_method(self, method_name, context):
        # 模拟工具调用
        action_map = {
            "decompose_contract": lambda: {"status": "ok", "sections": 8},
            "check_exclusivity": lambda: {"risk": True, "confidence": 0.92, "clause": "第5.2条"},
            "check_auto_renewal": lambda: {"risk": False, "confidence": 0.98},
            "generate_report": lambda: {"report_url": "/reports/123.pdf"}
        }
        action = action_map.get(method_name.split("_")[0], lambda: {"status": "unknown"})
        return action()
```

 **审计友好**：每一步都有日志记录，满足合规要求。


### 4. 与大模型深度集成：LLM 做“策略大脑”，HTN 做“执行引擎”

我们采用 **混合架构**，发挥各自优势：

```code
[用户输入] 
    ↓
LLM → 解析意图，生成高层任务（如“审查SaaS合同”）
    ↓
HTN Planner → 分解为结构化子任务流
    ↓
Executor → 调用工具（NLP提取、案例检索、规则匹配）
    ↓
Memory System ← 写入任务状态与风险证据
    ↓
LLM ← 汇总结果，生成自然语言报告
```

关键设计：

-  LLM **不直接执行任务**，只负责：
  - 意图识别
  - 自然语言总结
  - 用户交互澄清
-  所有关键决策由 HTN 控制，避免“幻觉调用”
-  工具调用通过注册表管理，确保安全



## 四、效果对比：从“黑盒”到“白盒执行”

| 步骤 | 传统 Agent | HTN Agent |
|------|------------|-----------|
| 1 | 直接输出报告，遗漏自动续约风险 | 分解任务，明确检查每一类条款 |
| 2 | 用户追问“为何没提？” → 无法解释 | 展示日志：“已检查，无风险（置信度 0.98）” |
| 3 | 发现新风险也无法调整流程 | 检测到排他性风险 → 自动插入“深度案例分析” |
| 4 | 报告无引用来源 | 每条结论关联具体条款与历史案例 |

## 五、执行


## 六、工程实践：构建可扩展的规划系统

### 1. 任务 DSL（领域特定语言）

定义声明式语法，降低维护门槛：

```python
# dsl/compliance_dsl.py
task("check_data_ownership") {
    precondition: has_clause("data_processing")
    steps: [
        extract_clause_content(),
        classify_data_type(),
        validate_policy(industry="SaaS")
    ]
    on_failure: escalate_to_human()
}
```

可构建 DSL 解析器，将上述语法转为 JSON 供 HTN 引擎使用。

### 2. 规划监控面板

```python
# monitor.py
class PlanningMonitor:
    def __init__(self):
        self.dashboard = []

    def log_step(self, task, status, confidence):
        entry = {
            "task": task,
            "status": status,
            "confidence": confidence,
            "timestamp": time.time()
        }
        self.dashboard.append(entry)
        print(f"[LOG] {task} | {status.upper()} | conf={confidence:.2f}")
```

支持：
- 实时追踪进度
- 设置超时中断
- 手动跳过环节

### 3. 与记忆系统联动

```python
# memory.py
class MemoryTransformer:
    def write(self, record):
        # 可写入向量数据库或日志系统
        print(f"[MEMORY] 记录: {record}")

# 执行后更新记忆
memory.write({
    "type": "risk_found",
    "task": "check_exclusivity",
    "clause": "第5.2条",
    "evidence": "限制客户使用竞品"
})
```

使后续任务能基于最新状态决策。

## 七、延伸场景：HTN 的更多应用

- 金融风控：贷款审批 → 动态触发反欺诈调查
- 医疗诊断：症状 → 检查 → 诊断 → 治疗建议
- 智能制造：设备报警 → 故障树分析 → 维修工单生成
- 自动驾驶：复杂路口 → 分解为“观察→让行→变道”子任务
  
HTN 是将**人类专家经验**转化为**可执行智能体行为**的最佳桥梁。

## 结语：让 Agent 拥有“系统性思维”

复杂任务的挑战不在单点智能，而在**全局协调与持续演进**。

通过 HTN 动态规划，我们实现了：

-  从“自由发挥” → “结构化执行”
-  从“不可控” → “可干预、可审计”
-  从“一次性推理” → “持续演进的任务流”